<a href="https://colab.research.google.com/github/mlcurva77/Assignments/blob/master/ML2019_ID10885123_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Loading NLTK
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

!pip install -q nltk==3.4.3

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
inGoogleColab = True
folderpath_base = '/content/drive/My Drive/Docs'

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
print(folderpath_base)

/content/drive/My Drive/Docs


In [4]:
import os
from nltk.corpus import PlaintextCorpusReader

#print(os.getcwd())
#corpus_root = os.getcwd() + "\Docs"
#print (corpus_root)

# .txt file names as file IDs
file_ids = ".*.txt"
#corpus = nltk.corpus.reader.PlaintextCorpusReader(corpus_root, file_ids,encoding='latin-1')
corpus = nltk.corpus.reader.PlaintextCorpusReader(folderpath_base, file_ids,encoding='latin-1')

#print(corpus.fileids())
print(corpus)

<PlaintextCorpusReader in '/content/drive/My Drive/Docs'>


In [5]:
# for-loop through file IDs and print out word count. 
for f in corpus.fileids():
    print(len(corpus.words(f)), f)

713 01_AFR1.txt
751 01_AFR2.txt
1360 01_AFR3.txt
906 01_AFR4.txt
399 02_EY1.txt
694 02_EY2.txt
3577 02_EY3.txt
12457 02_EY4.txt
6403 02_EY5.txt
7583 02_EY6.txt
14264 02_EY7.txt
821 02_EY8.txt
1460 03_EY1.txt
949 03_EY10.txt
1305 03_EY2.txt
1044 03_EY3.txt
515 03_EY4.txt
560 03_EY5.txt
865 03_EY6.txt
1371 03_EY7.txt
1245 03_EY8.txt
1201 03_EY9.txt
3603 04_EY1.txt
402 05_UNSW1.txt
1345 05_UQ1.txt
697 05_USYD1.txt
578 05_UTS1.txt
856 05_UWA1.txt


In [6]:
# Corpus size in number of words
print(len(corpus.words()))

67924


In [7]:
#tokenized_sent = corpus.sents()
#print(tokenized_sent)

tokenized_sent = corpus.sents()
print(tokenized_sent[0])

['US', 'business', 'leader', 'Kay', 'Koplovitz', 'forges', 'new', 'ground', 'for', 'female', 'entrepreneurs', 'by', 'Joanne', 'Gray', 'Women', 'in', 'science', 'and', 'technology', 'are', 'gaining', 'a', 'higher', 'profile', 'in', 'Australia', 'as', 'entrepreneurs', ',', 'says', 'Kay', 'Koplovitz', ',', 'a', 'leading', 'successful', 'businesswoman', 'in', 'the', 'US', 'and', 'founder', 'of', 'USA', 'Network', '.']


In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(tokenized_sent))

print(data_words[:2])

[['us', 'business', 'leader', 'kay', 'koplovitz', 'forges', 'new', 'ground', 'for', 'female', 'entrepreneurs', 'by', 'joanne', 'gray', 'women', 'in', 'science', 'and', 'technology', 'are', 'gaining', 'higher', 'profile', 'in', 'australia', 'as', 'entrepreneurs', 'says', 'kay', 'koplovitz', 'leading', 'successful', 'businesswoman', 'in', 'the', 'us', 'and', 'founder', 'of', 'usa', 'network'], ['her', 'successful', 'incubator', 'springboard', 'enterprises', 'launched', 'in', 'australia', 'five', 'years', 'ago', 'trains', 'women', 'founders', 'with', 'potentially', 'high', 'growth', 'businesses', 'to', 'raise', 'venture', 'capital', 'with', 'week', 'course', 'and', 'two', 'day', 'boot', 'camp', 'which', 'runs', 'this', 'week']]


In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[1]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['her', 'successful', 'incubator', 'springboard', 'enterprises', 'launched', 'in', 'australia', 'five', 'years_ago', 'trains', 'women', 'founders', 'with', 'potentially', 'high', 'growth', 'businesses', 'to', 'raise', 'venture_capital', 'with', 'week', 'course', 'and', 'two', 'day', 'boot', 'camp', 'which', 'runs', 'this', 'week']


In [12]:
from nltk.corpus import stopwords
stop_words=stopwords.words("english")
print(len(stop_words))

stop_words.extend(["can", "say","one","way","use","here","I",
                 "also","howev","tell","will","help","also",
                 "much","need","take","tend","even","us","US"
                 "like","particular","rather","said","EY","ey"
                 "get","well","make","ask","come","end",
                 "first","two","help","often","may",
                 "might","see","someth","thing","point",
                 "post","look","right","now","think","'ve ",
                 "'re ","anoth","put","set","new","good",
                 "want","sure","kind","larg","yes,","day","etc",
                 "quit","sinc","attempt","lack","seen","awar",
                 "littl","ever","moreov","though","found","abl",
                 "enough","far","earli","away","achiev","draw",
                 "last","never","brief","bit","entir","brief",
                 "great","lot","include","provide","year","new","help",
                 "includ","one","require","use","make","better",
                 "way","like","across","valu","provide","young","continu",
                 "tropfest","ernst","four","limit","and","however", "\x92", "\x93","\x94",
                 "\x95","\x96","\x97","\x98","\x99","the","in","because","includ","this","those","where"])
print(len(stop_words))
print(stop_words)

179
303
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', '

In [0]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['business', 'leader', 'kay', 'koplovitz', 'forge', 'ground', 'female', 'entrepreneur', 'gray', 'woman', 'science', 'technology', 'gain', 'high', 'profile', 'australia', 'entrepreneur', 'say', 'kay', 'koplovitz', 'lead', 'successful', 'businesswoman', 'founder', 'usa', 'network']]


In [15]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)]]


In [16]:
id2word[1]

'business'

In [17]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('australia', 1),
  ('business', 1),
  ('businesswoman', 1),
  ('entrepreneur', 2),
  ('female', 1),
  ('forge', 1),
  ('founder', 1),
  ('gain', 1),
  ('gray', 1),
  ('ground', 1),
  ('high', 1),
  ('kay', 2),
  ('koplovitz', 2),
  ('lead', 1),
  ('leader', 1),
  ('network', 1),
  ('profile', 1),
  ('say', 1),
  ('science', 1),
  ('successful', 1),
  ('technology', 1),
  ('usa', 1),
  ('woman', 1)]]

In [0]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [20]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.076*"business" + 0.044*"people" + 0.037*"idea" + 0.028*"opportunity" + '
  '0.025*"world" + 0.020*"scholarship" + 0.020*"award" + 0.016*"panel" + '
  '0.015*"benefit" + 0.014*"help"'),
 (1,
  '0.034*"datum" + 0.020*"event" + 0.020*"value" + 0.018*"competition" + '
  '0.016*"role" + 0.016*"get" + 0.015*"service" + 0.013*"development" + '
  '0.013*"use" + 0.012*"pay"'),
 (2,
  '0.042*"organisation" + 0.037*"research" + 0.036*"support" + 0.031*"provide" '
  '+ 0.026*"develop" + 0.024*"impact" + 0.016*"grow" + 0.015*"polson" + '
  '0.013*"similar" + 0.013*"agreement"'),
 (3,
  '0.033*"audit" + 0.024*"encourage" + 0.021*"report" + 0.018*"study" + '
  '0.015*"sustainable" + 0.014*"applicant" + 0.014*"working_world" + '
  '0.013*"cognitive" + 0.013*"application" + 0.013*"building"'),
 (4,
  '0.022*"recycling" + 0.021*"electrical" + 0.020*"waste" + 0.019*"equipment" '
  '+ 0.015*"pitch" + 0.015*"return" + 0.013*"national" + 0.012*"deliver" + '
  '0.011*"repair" + 0.011*"sydney"'),
 (

In [21]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.104756899369283

Coherence Score:  0.3953866657776465


In [0]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

In [0]:
import string
exclude = set(string.punctuation) 
print(exclude)

nopunctuation_word=[]
for w in tokenized_word:
    if w not in exclude:
        nopunctuation_word.append(w)
        
print("nopunctuation:",nopunctuation_word[:100])

In [0]:
from nltk.probability import FreqDist

fdist = FreqDist(nopunctuation_word)
print(fdist)
fdist.most_common(20)

In [0]:
import matplotlib.pyplot as plt
fdist.plot(30,cumulative=False)
plt.show()

In [0]:
lower_word = [word.lower() for word in nopunctuation_word]
print(lower_word[:100])

filtered_word=[]
for w in lower_word:
    if w not in stop_words:
        filtered_word.append(w)
#print("nopunctuation:",nopunctuation_word)
print("Filtered:",filtered_word)

In [0]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

stemmed_word=[]
for w in filtered_word:
    stemmed_word.append(ps.stem(w))

#print("filtered:",filtered_word)
print("Stemmed:",stemmed_word)

In [0]:
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

lm_word=[]
for w in stemmed_word:
    lm_word.append(lem.lemmatize(w,"v"))

print("Lemmatized:",lm_word[:100])

In [0]:
clean_word = [word.lower() for word in lm_word]
print(clean_word[:100])

In [0]:
fdist = FreqDist(clean_word)
print(fdist)
fdist.most_common(20)

import matplotlib.pyplot as plt
fdist.plot(30,cumulative=False)
plt.show()

In [0]:
from nltk.corpus import webtext
from nltk.probability import FreqDist
from wordcloud import WordCloud
import matplotlib.pyplot as plt
 
nltk.download('webtext')
#wt_words = webtext.words('testing.txt')  # Sample data
data_analysis = nltk.FreqDist(clean_word)
 
filter_words = dict([(m, n) for m, n in data_analysis.items() if len(m) > 1])
 
wcloud = WordCloud().generate_from_frequencies(filter_words)
 
# Plotting the wordcloud
plt.imshow(wcloud, interpolation="bilinear")
 
plt.axis("on")
(-0.5, 499.5, 299.5, -0.5)
plt.show()

In [0]:
dictionary = corpora.Dictionary(clean_word.tolist())
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break


In [0]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(clean_word, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[clean_word], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(bigram_mod[clean_word[3]])
print(trigram_mod[bigram_mod[clean_word[3]]])
print(bigram)